<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
!pip install tf-nightly-gpu
try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass
tf.enable_v2_behavior()
print(tf.__version__)

     |████████████████████████████████| 511.7MB 31kB/s 
     |████████████████████████████████| 2.9MB 52.4MB/s 
     |████████████████████████████████| 2.8MB 49.4MB/s 
     |████████████████████████████████| 460kB 61.4MB/s 
     |████████████████████████████████| 778kB 57.1MB/s 
ERROR: tensorflow 2.0.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
  Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0
2.2.0-dev20200218


In [ ]:
from __future__ import print_function

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

import numpy as np
import random
import sys
import os

In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
data_files = os.listdir('./shakespeare')

In [ ]:
# Read in Data

data = []

for file in data_files:
    if file[-3:] == 'txt':
        with open(f'./shakespeare/{file}', 'r', encoding='utf-8') as f:
            data.append(f.read())

In [7]:
len(data)

3

In [8]:
data[0][:500]

"ACT I\nSCENE I. Before LEONATO'S house.\nEnter LEONATO, HERO, and BEATRICE, with a Messenger\nLEONATO\nI learn in this letter that Don Peter of Arragon\ncomes this night to Messina.\nMessenger\nHe is very near by this: he was not three leagues off\nwhen I left him.\nLEONATO\nHow many gentlemen have you lost in this action?\nMessenger\nBut few of any sort, and none of name.\nLEONATO\nA victory is twice itself when the achiever brings\nhome full numbers. I find here that Don Peter hath\nbestowed much honour on a "

In [ ]:
# Encode data as characters
char_list = set()
for play in data:
    chars = set(play)
    char_list = char_list.union(chars)
    
char_list = list(char_list)

In [10]:
len(char_list)

64

In [ ]:
# Lookup tables
char_int = {c:i for i, c in enumerate(char_list)}
int_char = {i:c for i, c in enumerate(char_list)}

In [12]:
# Create the sequence data

maxlen = 100
step = 1

sequences = []
next_char = []

for play in data:
    encoded = [char_int[c] for c in play]
    for i in range(0, len(encoded) - maxlen, step):
        sequences.append(encoded[i : i + maxlen])
        next_char.append(encoded[i + maxlen])
        
print('Number of sequences: ', len(sequences))

Number of sequences:  419763


In [13]:
sequences[0][:50]

[31,
 22,
 19,
 2,
 23,
 36,
 20,
 22,
 11,
 13,
 11,
 2,
 23,
 30,
 2,
 44,
 48,
 51,
 0,
 32,
 48,
 2,
 41,
 11,
 12,
 13,
 31,
 19,
 12,
 9,
 20,
 2,
 6,
 0,
 62,
 28,
 48,
 30,
 36,
 11,
 14,
 52,
 48,
 32,
 2,
 41,
 11,
 12,
 13,
 31]

In [ ]:
# Create x and y

x = np.zeros((len(sequences), maxlen, len(char_list)), dtype=np.bool)
y = np.zeros((len(sequences), len(char_list)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char] = 1
        
    y[i, next_char[i]] = 1

In [15]:
x.shape, y.shape

((419763, 100, 64), (419763, 64))

In [25]:
# Build the model: a single LSTM
model = Sequential()
model.add(LSTM(512, input_shape=(maxlen, len(char_list)), dropout=0.2))
model.add(Dense(len(char_list), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 512)               1181696   
_________________________________________________________________
dense_2 (Dense)              (None, 64)                32832     
Total params: 1,214,528
Trainable params: 1,214,528
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    rand_play = random.randint(0, len(data) - 1)
    start_index = random.randint(0, len(data[rand_play]) - maxlen - 1)
    
    generated = ''
    
    sentence = data[rand_play][start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(1000):
        x_pred = np.zeros((1, maxlen, len(char_list)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [26]:
# Fit the model
model.fit(x, y,
          batch_size=512,
          epochs=50,
          callbacks=[print_callback],
          shuffle=True)

Train on 419763 samples
Epoch 1/50
419328/419763 [============================>.] - ETA: 0s - loss: 2.6689
----- Generating text after Epoch: 0
----- Generating with seed: "sed
With cheques as flatteries,--when they are seen abused.
Remember what I tell you.
OSWALD
Well, m"
sed
With cheques as flatteries,--when they are seen abused.
Remember what I tell you.
OSWALD
Well, melle dppand thut
Is turker on: hid drch'gceen ne?
SMay Culingir'd dingrtcen the Coid.
Fha nEwe wass the in ss, Ahdr dokr if y,
Forkmuse thes lis. Igo it all btance?
Tha MLIC IY ald hete a tas
. chssiof me speld anly hilk prrsels nxo
Falldw:ofstan s arsune, dis de beckniss his.
Thamk whtermnm,
Go richin, ladt men te, inst hich thesy Mrapstleernath: thit ghou the mrity . DENTDEMO
Yut bouw; cod ald atr
Thee un te and se r nt
who rbin she str ind no go. Iar be astenst:.EDINMLARTVant sean;'sm'sine.
Wh-spritpst sut.MGANR
II Beld whise thath inid
sayke bhue har ysu a, herd hevgnd
no whact eat the gsu.
If the weal:s hom ghisg

In [ ]:
import h5py

In [53]:
rand_play = random.randint(0, len(data) - 1)
start_index = random.randint(0, len(data[rand_play]) - maxlen - 1)
    
generated = ''
    
sentence = data[rand_play][start_index: start_index + maxlen]
    
sentence = "OTHELLO\nO my fair warrior!\n\nDESDEMONA\nMy dear Othello!\n\nOTHELLO\nIt gives me wonder great as my content"
sentence = sentence[:maxlen]
generated += sentence
    
print('----- Generating with seed: "' + sentence + '"')
print('-----------------')
sys.stdout.write(generated)
    
for _ in range(1000):
    x_pred = np.zeros((1, maxlen, len(char_list)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_int[char]] = 1
            
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = int_char[next_index]
        
    sentence = sentence[1:] + next_char
        
    sys.stdout.write(next_char)
    sys.stdout.flush()

----- Generating with seed: "OTHELLO
O my fair warrior!

DESDEMONA
My dear Othello!

OTHELLO
It gives me wonder great as my conte"
-----------------
OTHELLO
O my fair warrior!

DESDEMONA
My dear Othello!

OTHELLO
It gives me wonder great as my content To take thee and the candle: if he be eed,
When I copranny-upon your love her.
Exeunt

SCENE II. A church of the castle.
Enter GONERIL and EDMUND
CORNWALL
Give it your head; sir, our rnscintious mandard,
And sarest accusions rage.
Put some much sivest come now defair bare,
That till I brat I come on. Old savah
You kill my hup stater.
LODOVICO
The druncels five is one at graym of love,
And bid me fell that Doe mooks or judgment,
Nay, guending, sire when, binish'd, and eight I'ld saint: they were an indeenow: I am mightial, I
have losted, cross to tell him to meet for God shall beee
him that it come. I will get you ser in time
ouch of my chatber whiles here, it is smofe
and wit what will; bet o' the mine. 'twill spoke,'
pardon secl thy styl

In [ ]:
model.save('shakespeare_lstm_512.h5')

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN